In [20]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.utils import to_categorical
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

In [10]:
def cnn():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(7, activation='softmax'))
    model.compile(loss=keras.losses.binary_crossentropy, optimizer=keras.optimizers.Adam(0.0005), metrics=['accuracy'])
    return model

model, model_mask = cnn(), cnn()

In [11]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_accuracy', mode='max', verbose=0, patience=15, restore_best_weights=True)

In [12]:
fit_params = {'callbacks': [es],
              'epochs': 5,
              'batch_size': 32,
              'validation_split': .2,
              'verbose': 0}

split_params = {'split': 'one-split',
                'perturb': 0.03,
                'num_perm': 100,                
                'perturb_scale': 5,
                'min_inf': 100,
                'min_est': 1000,
                'ratio_method': 'fuse',
                'verbose': 1}

In [34]:
left_eye=np.load('../input/valid-fer/left_eye.npy')
left_eye=left_eye.astype(int)
valid_array=np.load('../input/valid-array/valid_array (4).npy')
valid_label=np.load('../input/valid-fer/valid_label.npy')
X=valid_array.astype('float32')/255
y=to_categorical(valid_label)

In [ ]:
Z=X.copy()
for i in range(len(valid_array)):
    x_left=left_eye[i,0]
    x_right=left_eye[i,1]
    y_up=left_eye[i,2]
    y_down=left_eye[i,3]
    Z[i,x_left:x_right+1,y_up:y_down+1,:]=0

In [16]:
from BBoxTest import split_test
cue = split_test(model=model, model_mask=model_mask, change='mask', eva_metric='zero-one',inf_feats=['left_eye'])

In [35]:
p_value_tmp = cue.ada_testing(X=X, Z=Z, y=y, ratio=0.2,fit_params=fit_params, split_params=split_params)